In [2]:
from astroquery.mast import Observations
from astroquery.exceptions import RemoteServiceError
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt

# List of RA and Dec coordinates for your targets (in degrees)
coordinates = [
    (191.29222, 43.81053),  # Example: J1245+4348
    (110.5, 58.00001),  # Example: J1100+5800
    (132.845, 44.75),  # Example: J1328+4445
    # Add more targets here...
]

# Function to query MAST and retrieve spectra
def get_spectra_from_mast(ra, dec):
    co = SkyCoord(ra=ra, dec=dec, unit="deg")
    try:
        # Query MAST for the spectrum using the Observations class
        print(f"Querying MAST for RA={ra}, Dec={dec}...")
        
        # Query for observations in the area
        obs = Observations.query_region(co, radius=0.1)
        
        if len(obs) > 0:
            print(f"Found {len(obs)} observations for RA={ra}, Dec={dec}.")
            
            # Look through the observations and try to find a spectrum
            for observation in obs:
                # Check if the observation contains spectrum data (it might be in a different format)
                if 'spectrum' in observation['dataProductType']:
                    # Download the spectrum data (e.g., FITS file)
                    print(f"Downloading spectrum for RA={ra}, Dec={dec}.")
                    data = Observations.get_product_list(observation)
                    
                    # Find the spectrum data in the product list
                    spectrum_data = None
                    for product in data:
                        if product['dataProductType'] == 'spectrum':
                            spectrum_data = product
                            break
                    
                    if spectrum_data is not None:
                        # Download the spectrum file
                        Observations.download_products([spectrum_data])
                        
                        # Extract and plot the spectrum (assuming FITS format for simplicity)
                        # You might need to adjust this part based on the actual format of the spectrum data
                        from astropy.io import fits
                        spectrum_file = spectrum_data['fileName']
                        with fits.open(spectrum_file) as hdul:
                            spectrum = hdul[1].data  # Assuming the spectrum is in the second HDU
                            wavelength = spectrum['WAVELENGTH']  # Adjust the column name as needed
                            flux = spectrum['FLUX']  # Adjust the column name as needed
                        
                        # Plot the spectrum
                        plt.figure(figsize=(10, 5))
                        plt.plot(wavelength, flux, label=f"Spectrum for RA={ra}, Dec={dec}")
                        plt.xlabel("Wavelength (Å)")
                        plt.ylabel("Flux (10⁻¹⁷ erg/s/cm²/Å)")
                        plt.title(f"Spectrum for RA={ra}, Dec={dec}")
                        plt.legend()
                        plt.show()
                        return spectrum_data
        else:
            print(f"No observations found for RA={ra}, Dec={dec}.")
            return None
    except RemoteServiceError:
        print(f"Error querying MAST for RA={ra}, Dec={dec}.")
        return None

# Loop through the list of targets and query MAST
for ra, dec in coordinates:
    spectrum = get_spectra_from_mast(ra, dec)
    
    if spectrum is not None:
        print(f"Successfully retrieved and plotted the spectrum for RA={ra}, Dec={dec}.")


Querying MAST for RA=191.29222, Dec=43.81053...
Found 31 observations for RA=191.29222, Dec=43.81053.
Spectrum found for RA=191.29222, Dec=43.81053.
Querying MAST for RA=110.5, Dec=58.00001...
Found 53 observations for RA=110.5, Dec=58.00001.
Spectrum found for RA=110.5, Dec=58.00001.
Querying MAST for RA=132.845, Dec=44.75...
Found 367 observations for RA=132.845, Dec=44.75.


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>